In [1003]:
"""
Скрипт для обработки общйе таблицы Омского ЦОПП. 
"""

'\nСкрипт для обработки общйе таблицы Омского ЦОПП. \n'

In [1004]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import tkinter
import sys
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time

pd.options.mode.chained_assignment = None  # default='warn'
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import copy
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import random

In [1005]:
def check_data(cell):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    if cell is np.nan:
        return 0
    if cell.isdigit():
        return int(cell)
    else:
        return 0

In [1006]:
def group_text_value(value):
    """
    функция для группировки текстовых данных, группирует только заполненные строки
    """
    tmp_set = set(value.tolist()) # создаем множество
    tmp_set.discard('Не заполнено')
    tmp_set.discard('0')
    return
    print(tmp_set)


In [1007]:
def check_sameness_range(df:pd.DataFrame,names_columns:list,border,amendment):
    
    """    
    Функция для проверки единнобразия значений в колонке по диапазонам
    """
    # получаем поправки для того чтобы диапазон ошибки указывался корректно
    first_correct = amendment[0]
    second_correct = amendment[1] 
    _error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки']) # датафрейм для ошибок
    for column in names_columns:
        _temp_set = set(df[column].tolist())
        if len(_temp_set) != 1:
            temp_error_df = pd.DataFrame(data=[[f'Диапазон строк {border + first_correct} - {border+second_correct},Колонка {column}',
                                        'В указанном диапазоне и колонке  должны быть одинаковые значения во избежание ошибок при подсчете,если указан диапазон вне таблицы то удалите строки этого диапазона']],
                                 columns=['Строка или колонка с ошибкой',
                                          'Описание ошибки'])
            _error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
    return _error_df
        

In [1008]:
def check_error_omsk(df:pd.DataFrame,border,amendment):
    """
    Функция для проверки на ошибки
    """

    error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки']) # датафрейм для ошибок
    quantity_range = df.shape[0] // 5 #  получаем количество проходов которые нужно пройти
    sameness_columns = ['1','2','3'] # список колонок которые нужно проверить на единообразие внутри диапазона
    for i in range(quantity_range):
        temp_df = df.iloc[border:border+5,:]
        error_sameness_df = check_sameness_range(temp_df.copy(),sameness_columns,border,amendment) # проверка на единообразие
        error_df = pd.concat([error_df, error_sameness_df], axis=0, ignore_index=True)       
              
        
        
        
        border += 5
        
        
        
    
    
    
    
    
    return error_df
        
        
        
        
    
    
    
    

In [1009]:
path_data_omsk = 'data/Омск/Трудоустройство_1_07_2023_выпуск_2022.xlsx'
path_to_end_folder = 'data'
# создаем датафрейм для регистрации ошибок
error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки'])

In [1010]:
# загружаем файл
df = pd.read_excel(f'{path_data_omsk}',dtype=str,skiprows=5)

In [1011]:
df=df.iloc[:,:34] # убираем строки проверки

In [1012]:

# заменяем названия колонок
df.columns = list(map(str,range(1,df.shape[1]+1)))

In [1013]:
df.shape

(2030, 34)

In [1014]:
# Проверяем на количество строк, должно быть кратно 5
if df.shape[0] % 5 !=0:
    temp_error_df = pd.DataFrame(data=[['',
                                        'Количество строк не кратно 5 !!! Возможно пропущены строки с данными или есть лишние строки в конце таблицы']],
                                 columns=['Строка или колонка с ошибкой',
                                          'Описание ошибки'])
    error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)

In [1015]:
# Проверяем ошибки
tup_correct = (7, 11)  # создаем кортеж  с поправками
check_error_df= check_error_omsk(df.copy(),0,tup_correct)
error_df = pd.concat([error_df, check_error_df], axis=0, ignore_index=True)


In [1016]:
df.head(1)

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,"БПОУ ""Седельниковский агропромышленный техникум""",23.01.03,Автомеханик,01,Всего,22,5,1,"1 чел -ПАО ""ОПК-Сатурн"" ОМКБ",4,...,0,0,0,1,0,0,0,0,0,Опрос-6 чел


In [1017]:
text_df = df[['2','3','4','5','9','34']]

In [1018]:
text_df

,2,3,4,5,9,34
0,23.01.03,Автомеханик,01,Всего,"1 чел -ПАО ""ОПК-Сатурн"" ОМКБ",Опрос-6 чел
1,23.01.03,Автомеханик,02,Лица с ограниченными возможностями здоровья,NaN,NaN
2,23.01.03,Автомеханик,03,из них (из строки 02): инвалиды и дети-инвалиды,NaN,NaN
3,23.01.03,Автомеханик,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,NaN,NaN
4,23.01.03,Автомеханик,05,Имеют договор о целевом обучении,NaN,NaN
...,...,...,...,...,...,...
2025,49.02.01,Физическая культура,01,Всего,NaN,NaN
2026,49.02.01,Физическая культура,02,Лица с ограниченными возможностями здоровья,NaN,NaN
2027,49.02.01,Физическая культура,03,из них (из строки 02): инвалиды и дети-инвалиды,NaN,NaN
2028,49.02.01,Физическая культура,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,NaN,NaN


In [1019]:
df.drop(columns=['1','9','34'],inplace=True) #Удаляем лишние колонки мешающие группировке

In [1020]:
columns_to_apply = ['6','7','8','10','11','12','13','14','15','16','17','18','19','20','21','22','23'
                    ,'24','25','26','27','28','29','30','31','32','33']

In [1021]:
df[columns_to_apply] = df[columns_to_apply].applymap(check_data)

In [1022]:
df

,2,3,4,5,6,7,8,10,11,12,...,24,25,26,27,28,29,30,31,32,33
0,23.01.03,Автомеханик,01,Всего,22,5,1,4,2,0,...,0,0,0,0,1,0,0,0,0,0
1,23.01.03,Автомеханик,02,Лица с ограниченными возможностями здоровья,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23.01.03,Автомеханик,03,из них (из строки 02): инвалиды и дети-инвалиды,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,23.01.03,Автомеханик,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,23.01.03,Автомеханик,05,Имеют договор о целевом обучении,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,49.02.01,Физическая культура,01,Всего,29,3,0,2,2,4,...,0,0,0,0,0,0,0,0,0,0
2026,49.02.01,Физическая культура,02,Лица с ограниченными возможностями здоровья,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2027,49.02.01,Физическая культура,03,из них (из строки 02): инвалиды и дети-инвалиды,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2028,49.02.01,Физическая культура,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1023]:
df.groupby(['2','3','4','5']).agg(sum) # проводим группировку
df = df.reset_index() # извлекаем мультииндекс
df.drop(columns='index',inplace=True)

In [1024]:
df

,2,3,4,5,6,7,8,10,11,12,...,24,25,26,27,28,29,30,31,32,33
0,23.01.03,Автомеханик,01,Всего,22,5,1,4,2,0,...,0,0,0,0,1,0,0,0,0,0
1,23.01.03,Автомеханик,02,Лица с ограниченными возможностями здоровья,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23.01.03,Автомеханик,03,из них (из строки 02): инвалиды и дети-инвалиды,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,23.01.03,Автомеханик,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,23.01.03,Автомеханик,05,Имеют договор о целевом обучении,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,49.02.01,Физическая культура,01,Всего,29,3,0,2,2,4,...,0,0,0,0,0,0,0,0,0,0
2026,49.02.01,Физическая культура,02,Лица с ограниченными возможностями здоровья,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2027,49.02.01,Физическая культура,03,из них (из строки 02): инвалиды и дети-инвалиды,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2028,49.02.01,Физическая культура,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1025]:
text_df

,2,3,4,5,9,34
0,23.01.03,Автомеханик,01,Всего,"1 чел -ПАО ""ОПК-Сатурн"" ОМКБ",Опрос-6 чел
1,23.01.03,Автомеханик,02,Лица с ограниченными возможностями здоровья,NaN,NaN
2,23.01.03,Автомеханик,03,из них (из строки 02): инвалиды и дети-инвалиды,NaN,NaN
3,23.01.03,Автомеханик,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,NaN,NaN
4,23.01.03,Автомеханик,05,Имеют договор о целевом обучении,NaN,NaN
...,...,...,...,...,...,...
2025,49.02.01,Физическая культура,01,Всего,NaN,NaN
2026,49.02.01,Физическая культура,02,Лица с ограниченными возможностями здоровья,NaN,NaN
2027,49.02.01,Физическая культура,03,из них (из строки 02): инвалиды и дети-инвалиды,NaN,NaN
2028,49.02.01,Физическая культура,04,Инвалиды и дети-инвалиды (кроме учтенных в стр...,NaN,NaN


In [1026]:
text_df.fillna('Не заполнено',inplace=True)

In [1027]:
group_text_df = text_df.groupby(['2','3','4','5']).agg({'9':group_text_value,'34':group_text_value})
#group_text_df = text_df.groupby(['2','3','4','5']).agg({'9':group_text_value,'34':';'.join})
group_text_df = group_text_df.reset_index()


['2 чел - АО "Омсктрансмаш",  1 чел. - Филиал "ОДК" - "ОМО им. П. И. Баранова"0', '5 чел- ЦК ТАО']
['0', 'Не заполнено']
['0', 'Не заполнено']
['0', 'Не заполнено']
['Не заполнено', 'Не заполнено']
['0']
['0']
['0']
['0']
['0']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['0']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['0']
['0']
['0']
['0']
['0']
['Не заполнено', 'Не заполнено']
['Не заполнено', 'Не заполнено']
['Не заполнено', 'Не заполнено']
['Не заполнено', 'Не заполнено']
['Не заполнено', 'Не заполнено']
['0', 'Не заполнено', 'Не заполнено', 'Не заполнено']
['0', 'Не заполнено', 'Не заполнено', 'Не заполнено']
['0', 'Не заполнено', 'Не заполнено', 'Не заполнено']
['0', 'Не заполнено', 'Не заполнено', 'Не заполнено']
['0', 'Не заполнено', 'Не заполнено', 'Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполнено']
['Не заполне

In [1028]:
group_text_df[['9','34']] =group_text_df[['9','34']].applymap(lambda x:x.replace('Не заполнено',''))
group_text_df[['9','34']] =group_text_df[['9','34']].applymap(lambda x:x.replace('0',''))


AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
group_text_df

In [ ]:
group_text_df.to_excel('trs.xlsx',index=False)

In [ ]:
error_df